In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Relatório sobre a Retomada Presencial</em><p>

<p>Os dados foram coletados a partir de um formulário com as seguintes perguntas:</p>

<ul>
    <li>Nome</li>
    <li>Curso</li>
    <li>Turno</li>
    <li>Sente-se seguro/confortável para retornar às aulas presenciais?</li>
    <li>Você precisa fazer uso de transporte público para chegar a Fatec?</li>
    <li>Você mora com alguém do grupo de risco?</li>
    <li>Deixe-nos saber sobre sua situação vacinal:</li>
    <li>Tem previsão para tomar a 2a. dose? (Apenas para respondentes de vacinados parcialmente)</li>
</ul>
</div>
"""

In [5]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
    border-bottom: 2px solid rgb(148,39,44);
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}

.logo {
    max-width: 50%;
    height: auto;
    display: block;
    margin-left: auto;
    margin-right: auto;
}

.top {
    border-bottom: 5px solid rgb(148,39,44);
}

.focus {
    font-size: xlarger;
}
</style>
""")

In [1]:
class App:
    def __init__(self):
        self.set_general_config()
        self.load_data()
        self.create_container()
        self._update_app()
    
    def set_general_config(self):
        self._width = 12
        self._height = 6
        sns.set(rc = {'figure.figsize':(self._width,self._height)})
        sns.set(font_scale=1.5)
        sns.set_style("whitegrid")
    
    def load_data(self):
        self.df_estudantes = pd.read_excel('retomada_presencial/Retorno Presencial.xlsx')
        self.df_respostas = pd.read_excel('retomada_presencial/Retomada Presencial das Aulas.xlsx')
        self.data_prepare()
        
    def data_prepare(self):
        def status_imunizacao(v):
            if v in ['Tomei dose única', 'Tomei duas doses']:
                return 'Completa'
            elif v == 'Tomei somente a 1a. dose':
                return 'Parcial'
            return 'Não iniciada' 
        siglas_curso = {
            'Análise e Desenvolvimento de Sistemas': 'ADS',
            'Gestão de Negócios e Inovação': 'GNI',
            'Sistemas Biomédicos': 'SBM'
        }
        self.df_respostas['Curso (Sigla)'] = self.df_respostas.loc[:, 'Curso'].apply(lambda v: siglas_curso[v])
        self.df_respostas['Imunização'] = self.df_respostas.loc[:, 'Deixe-nos saber sobre sua situação vacinal:'].apply(status_imunizacao)
        self.df_respostas_disciplinas = pd.merge(self.df_estudantes, self.df_respostas, how='left', left_on='Email Cadastrado', right_on='Email')
        self.df_respostas_disciplinas = self.df_respostas_disciplinas.rename({'Turno_x': 'Turno', 'Curso_x': 'Curso'}, axis=1)

    def show_values_on_bars(self, ax, offset=25, orientation='H'):
        for p in ax.patches:
            if orientation == 'H':
                ax.annotate("{:.1f}".format(p.get_width()),
                           (p.get_width(), p.get_y() + p.get_height() / 2.),
                           ha='center', va='center',
                           xytext=(offset, 0),
                           textcoords='offset points')
            else:
                ax.annotate("{:.1f}".format(p.get_height()),
                           (p.get_x() + p.get_width() / 2., p.get_height()),
                           ha='center', va='center',
                           xytext=(0, offset),
                           textcoords='offset points')

    def get_percentual_question_course(self, qr, curso):
        filtro = (self.df_respostas_disciplinas.loc[:, 'Curso'] == curso)
        df_ = self.df_respostas_disciplinas.loc[filtro]
        df_perc = df_.loc[:, ['Disciplina', 'Turno', 'ID', qr]].groupby(['Turno', 'Disciplina', qr]).count().rename({'ID': '%'}, axis=1)
        return df_perc / df_perc.groupby(level=1).sum() * 100
    
    # Plot functions
    def plot_count_questao(self, questao, y='Curso (Sigla)', horder=None, loc='upper left'):
        g = sns.countplot(data=self.df_respostas, hue=questao, y=y, hue_order=horder)
        self.show_values_on_bars(g)
        g.set_xlabel('Quantidade')
        plt.legend(bbox_to_anchor=(1.04,1), loc=loc)
        plt.tight_layout()
        plt.show()

    def plot_percentual_question(self, qr):
        tot = self.df_respostas.loc[:, 'ID'].count()
        df_ = self.df_respostas.loc[:, [qr, 'ID']].groupby(qr).agg(lambda x: 100*len(x)/tot).rename({'ID': '%'},axis=1).reset_index()
        g = sns.catplot(kind='bar', x=qr, y='%', data=df_, height=self._height, aspect=self._width/self._height)
        g.ax.set_ylabel('%')
        self.show_values_on_bars(g.ax, orientation='V')
        
    def plot_disciplina(self, qr,curso,horder=None):
        filtro = (self.df_respostas_disciplinas.loc[:, 'Curso'] == curso)
        df_ = self.df_respostas_disciplinas.loc[filtro]
        if len(df_.loc[:, 'Turno'].unique()) > 1:
            g = sns.catplot(kind='count', y='Disciplina', hue=qr, data=df_, hue_order=horder, col='Turno')
            for ax in g.axes_dict.values():
                ax.set_xlabel('Quantidade')
        else:
            g = sns.countplot(y='Disciplina', hue=qr, data=df_, hue_order=horder)
            g.set_xlabel('Quantidade')

    def plot_disciplina_perc(qr,curso,horder=None):
        df_ = self.get_perc_disciplina_questao(qr, curso)
        df_ = df_.reset_index()
        if len(df_.loc[:, 'Turno'].unique()) > 1:
            g = sns.catplot(kind='bar', y='Disciplina', x='%', hue=qr, data=df_, hue_order=horder, col='Turno')

    def _create_plot_how_many(self):
        g = sns.countplot(data=self.df_respostas, y='Curso (Sigla)')
        self.show_values_on_bars(g)
        g.set_title('Respostas por Curso')
        g.set_xlabel('Quantidade')
    
    def _create_plot_how_many_shift(self):
        g = sns.countplot(data=self.df_respostas, x='Turno')
        self.show_values_on_bars(g, orientation='V')
        g.set_title('Respostas por Turno')
        g.set_ylabel('Quantidade')
        
    def _create_plot_how_many_night_shift(self):
        g = sns.countplot(data=self.df_respostas.loc[self.df_respostas.loc[:, 'Turno'] == 'Noite'], y='Curso (Sigla)')
        g.set_title('Respostas por Curso Noturno')
        self.show_values_on_bars(g)
        g.set_xlabel('Quantidade')
    
    def _plot_disciplina_perc(self, qr,curso,horder=None):
        df_ = self.get_percentual_question_course(qr, curso)
        df_ = df_.reset_index()
        if len(df_.loc[:, 'Turno'].unique()) > 1:
            g = sns.catplot(kind='bar', y='Disciplina', x='%', hue=qr, data=df_, hue_order=horder, col='Turno')
        else:
            g = sns.barplot(y='Disciplina', x='%', hue=qr, data=df_, hue_order=horder)        
    
    #####
    # Q:Sente-se seguro/confortável para retornar às aulas presenciais?
    ####
    def _create_plot_q1_perc(self):
        self.plot_percentual_question('Sente-se seguro/confortável para retornar às aulas presenciais?')
    
    def _create_plot_q1_curso(self):
        self.plot_count_questao(questao = 'Sente-se seguro/confortável para retornar às aulas presenciais?', horder=['Não', 'Sim'])
    
    def _create_plot_q1_turno(self):
        self.plot_count_questao(questao = 'Sente-se seguro/confortável para retornar às aulas presenciais?', y='Turno', horder=['Não', 'Sim'])
    
    #####
    # Q:Você precisa fazer uso de transporte público para chegar a Fatec?
    ####
    def _create_plot_q2_perc(self):
        self.plot_percentual_question('Você precisa fazer uso de transporte público para chegar a Fatec?')
    
    def _create_plot_q2_curso(self):
        self.plot_count_questao(questao = 'Você precisa fazer uso de transporte público para chegar a Fatec?', horder=['Não', 'Sim'])
    
    def _create_plot_q2_turno(self):
        self.plot_count_questao(questao = 'Você precisa fazer uso de transporte público para chegar a Fatec?', y='Turno', horder=['Não', 'Sim'])

    #####
    # Q:Você mora com alguém do grupo de risco?
    ####
    def _create_plot_q3_perc(self):
        self.plot_percentual_question('Você mora com alguém do grupo de risco?')
    
    def _create_plot_q3_curso(self):
        self.plot_count_questao(questao = 'Você mora com alguém do grupo de risco?', horder=['Não', 'Sim'])
    
    def _create_plot_q3_turno(self):
        self.plot_count_questao(questao = 'Você mora com alguém do grupo de risco?', y='Turno', horder=['Não', 'Sim'])

    #####
    # Q:Deixe-nos saber sobre sua situação vacinal:
    ####
    def _create_plot_q4_perc(self):
        self.plot_percentual_question('Imunização')
    
    def _create_plot_q4_curso(self):
        self.plot_count_questao(questao = 'Imunização')
    
    def _create_plot_q4_turno(self):
        self.plot_count_questao(questao = 'Imunização', y='Turno')

    #####
    # Q:Tem previsão para tomar a 2a. dose?
    ####
    def _create_plot_q5_perc(self):
        self.plot_percentual_question('Tem previsão para tomar a 2a. dose?')
    
    def _create_plot_q5_curso(self):
        self.plot_count_questao(questao = 'Tem previsão para tomar a 2a. dose?')
    
    def _create_plot_q5_turno(self):
        self.plot_count_questao(questao = 'Tem previsão para tomar a 2a. dose?', y='Turno')

    #####
    # Q: Imunização pra cada disciplina/curso
    #####
    def _create_plot_imunizacao(self, curso):
        self._plot_disciplina_perc('Imunização',curso)
    
    def change_DrpCourse(self, drp_chg):
        self._plot_container_imuni_course.clear_output(wait=True)
        with self._plot_container_imuni_course:
            self. _create_plot_imunizacao(self.drp_course.value)
            plt.show()
    
    def create_container(self):
        self.drp_course = widgets.Dropdown(
            options=self.df_respostas_disciplinas.loc[:, 'Curso'].unique(),
            description='Curso:',
            disabled=False
        )
        self.drp_course.observe(self.change_DrpCourse)
        self._plot_containers = [widgets.Output() for _ in range(18)]
        self._plot_container_imuni_course = widgets.Output()
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<div class="top"><img class="logo" src="images/logo_fatecrp_cps_colorido_fundobranco.png"/></div>'
                    '<h1>Retomada Presencial</h1>'
                    '<h2 class="app-subtitle"><a href="www.fatecrp.edu.br">fatecrp.edu.br</a></h2>'
                ), layout=widgets.Layout(margin='0 0 0 0')
            ),
            widgets.HBox([
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 0'))
            ]),
            widgets.VBox([
                widgets.HTML('<h2 class="app-subtitle">Respondentes</h2>', layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HTML('<p><span class="focus">{}/{}</span> estudantes responderam a pesquisa até o momento</p>'.format(self.df_respostas.loc[:, 'ID'].count(),self.df_estudantes.loc[:, 'Email Cadastrado'].unique().shape[0]), layout=widgets.Layout(margin='0 0 0 2em')),
                widgets.HBox([
                    self._plot_containers[1]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[0],
                    self._plot_containers[2]
                ], layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML('<h2 class="app-subtitle">Sente-se seguro/confortável para retornar às aulas presenciais?</h2>', layout=widgets.Layout(margin='0 0 0 2em')),
                widgets.HBox([
                    self._plot_containers[3]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[4],
                    self._plot_containers[5],
                ],layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML('<h2 class="app-subtitle">Você precisa fazer uso de transporte público para chegar a Fatec?</h2>', layout=widgets.Layout(margin='0 0 0 2em')),
                widgets.HBox([
                    self._plot_containers[6]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[7],
                    self._plot_containers[8],
                ], layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML('<h2 class="app-subtitle">Você mora com alguém do grupo de risco?</h2>', layout=widgets.Layout(margin='0 0 0 2em')),
                widgets.HBox([
                    self._plot_containers[9]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[10],
                    self._plot_containers[11],
                ], layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML('<h2 class="app-subtitle">Deixe-nos saber sobre sua situação vacinal:</h2>', layout=widgets.Layout(margin='0 0 0 2em')),
                widgets.HBox([
                    self._plot_containers[12]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[13],
                    self._plot_containers[14],
                ], layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML(
                    (
                        '<h2 class="app-subtitle">Tem previsão para tomar a 2a. dose?</h2>'
                        '<h3>Apenas para os parcialmente vacinados</h3>'
                    ), layout=widgets.Layout(margin='0 0 0 2em')
                ),
                widgets.HBox([
                    self._plot_containers[15]
                ], layout=widgets.Layout(margin='0 0 2em 2em')),
                widgets.HBox([
                    self._plot_containers[16],
                    self._plot_containers[17],
                ], layout=widgets.Layout(margin='0 0 4em 0')),
                widgets.HTML(
                    (
                        '<h2 class="app-subtitle">Imunização entre as Disciplinas</h2>'
                    ), layout=widgets.Layout(margin='0 0 0 2em')
                ),
                widgets.VBox([
                    self.drp_course,
                    self._plot_container_imuni_course
                ], layout=widgets.Layout(margin='0 0 2em 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
    
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):            
        pfunc = [
            self._create_plot_how_many,
            self._create_plot_how_many_shift,
            self._create_plot_how_many_night_shift,
            self._create_plot_q1_perc,
            self._create_plot_q1_curso,
            self._create_plot_q1_turno,
            self._create_plot_q2_perc,
            self._create_plot_q2_curso,
            self._create_plot_q2_turno,
            self._create_plot_q3_perc,
            self._create_plot_q3_curso,
            self._create_plot_q3_turno,
            self._create_plot_q4_perc,
            self._create_plot_q4_curso,
            self._create_plot_q4_turno,            
            self._create_plot_q5_perc,
            self._create_plot_q5_curso,
            self._create_plot_q5_turno            
        ]
        for pf, pc in zip(pfunc,self._plot_containers):
            pc.clear_output(wait=True)
            with pc:
                pf()
                plt.show()

In [17]:
app = App()
app.container